<a href="https://colab.research.google.com/github/PaolaMaribel18/RI_2024a/blob/main/week4/taller02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 02: Modelo de Espacio Vectorial

## Objetivo
 Comprender y aplicar el Modelo de Espacio Vectorial para el ranking de documentos
 basado en la similitud de contenido con una consulta dada.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step 1: Data Preprocessing

Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.
Write a function to load and preprocess the text documents from a specified directory. This step involves reading each file, converting the text to lowercase for uniform processing, and storing the results in a dictionary.

In [38]:
import os
import numpy as np
import re

# Define the path to the directory containing the text files
CORPUS_DIR = '/content/drive/MyDrive/ri_2024a/week4/data'

# Diccionario para almacenar los textos de los documentos
documents_text = {}

dictionary = {}

# Recorre los documentos y almacena sus textos
for filename in os.listdir(CORPUS_DIR):
    if filename.endswith('.txt'):
        file_path = os.path.join(CORPUS_DIR, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read().lower()  # Lee y convierte a minúsculas
            text = re.sub(r'[.,:;]', '', text)
            text = re.sub(r'[^a-zA-ZñÑáéíóú\s]', '', text)  # Elimina caracteres no alfanuméricos
            documents_text[filename] = text
            # Construye el diccionario de términos
            words = text.split()
            for word in words:
                dictionary[word] = True

In [39]:
# Crear una matriz vacía para almacenar los conteos de términos en documentos
term_document_matrix = np.zeros((len(dictionary), len(documents_text)), dtype=int)

# Mapear el índice de términos
term_index_mapping = {term: idx for idx, term in enumerate(dictionary)}

# Llenar la matriz con los conteos de términos en documentos
for doc_idx, (doc_name, doc_text) in enumerate(documents_text.items()):
    words = doc_text.split()
    for word in words:
        if word in term_index_mapping:
            term_idx = term_index_mapping[word]
            term_document_matrix[term_idx, doc_idx] += 1


In [40]:
print(term_document_matrix)

[[2 0 0 ... 0 1 0]
 [4 0 1 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [41]:
import csv

# Define el nombre del archivo CSV
csv_file = '/content/drive/MyDrive/ri_2024a/week4/term_document_matrix.csv'

# Guarda la matriz en el archivo CSV
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    # Escribe la primera fila con los nombres de los documentos
    writer.writerow(['Term'] + list(documents_text.keys()))
    # Escribe las filas con los conteos de términos en documentos
    for term, term_counts in zip(dictionary.keys(), term_document_matrix):
        writer.writerow([term] + list(term_counts))

In [42]:
import pandas as pd

# Lee el archivo CSV en un DataFrame
df = pd.read_csv(csv_file)

# Muestra el DataFrame
print(df)

            Term  t3.txt  t5.txt  t2.txt  t1.txt  t4.txt  t6.txt  t7.txt  \
0          museo       2       0       0       0       0       0       0   
1            del       4       0       1       1       2       0       1   
2          prado       2       0       0       0       0       0       0   
3             el       2       1       3       0       3       0       3   
4             es       1       1       2       0       1       2       1   
..           ...     ...     ...     ...     ...     ...     ...     ...   
370     especies       0       0       0       0       0       0       0   
371   amenazadas       0       0       0       0       0       0       0   
372  proporciona       0       0       0       0       0       0       0   
373    educativa       0       0       0       0       0       0       0   
374  entretenida       0       0       0       0       0       0       0   

     t10.txt  t9.txt  ...  t12.txt  t11.txt  t14.txt  t18.txt  t17.txt  \
0          0 

In [43]:
# Lista de líneas
lineas = [
    "Historia medieval de las ciudades europeas",
    "Principales destinos turísticos en Europa",
    "Influencia de la realeza en la cultura europea",
    "Importancia de los parques urbanos en las ciudades",
    "Gastronomía típica en capitales europeas",
    "Eventos deportivos icónicos en Europa",
    "Celebraciones de Navidad en ciudades europeas",
    "Museos de arte importantes en Europa",
    "Efectos de la Segunda Guerra Mundial en ciudades europeas",
    "Arquitectura histórica en ciudades europeas",
    "Clubes de fútbol famosos y sus estadios en Europa",
    "Transformación urbana y regeneración de espacios",
    "Tradición cervecera en ciudades europeas",
    "Mercados y comercio tradicional en Europa",
    "Centros de arte y creatividad en ciudades modernas",
    "Actividades de ocio y entretenimiento en ciudades metropolitanas",
    "Conservación de la naturaleza y la vida silvestre en zonas urbanas",
    "Planificación de eventos culturales y festivales en ciudades",
    "Desarrollo del transporte y la infraestructura urbana",
    "Impacto de la tecnología en la vida urbana"
]

# Crear archivos .txt para cada línea
for i, linea in enumerate(lineas):
    nombre_archivo = f"q{i+1}.txt"
    with open(nombre_archivo, "w") as archivo:
        archivo.write(linea)


In [44]:
# Crear una matriz vacía para almacenar los conteos de términos en las líneas
term_query_matrix = np.zeros((len(dictionary), len(lineas)), dtype=int)

# Recorre las líneas y llena la matriz con los conteos de términos
for line_idx, line in enumerate(lineas):
    words = line.lower().split()  # Convertir a minúsculas y dividir en palabras
    for word in words:
        if word in term_index_mapping:
            term_idx = term_index_mapping[word]
            term_query_matrix[term_idx, line_idx] += 1


In [45]:
print(term_query_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [46]:
# Define el nombre del archivo CSV
csv_file_q = '/content/drive/MyDrive/ri_2024a/week4/term_query_matrix.csv'

# Guarda la matriz en el archivo CSV
with open(csv_file_q, 'w', newline='') as file:
    writer = csv.writer(file)
    # Escribe la primera fila con los nombres de los documentos
    writer.writerow(['Term'] + list(documents_text.keys()))
    # Escribe las filas con los conteos de términos en documentos
    for term, term_counts in zip(dictionary.keys(), term_document_matrix):
        writer.writerow([term] + list(term_counts))

In [47]:
df_q = pd.read_csv(csv_file_q)
print(df_q)

            Term  t3.txt  t5.txt  t2.txt  t1.txt  t4.txt  t6.txt  t7.txt  \
0          museo       2       0       0       0       0       0       0   
1            del       4       0       1       1       2       0       1   
2          prado       2       0       0       0       0       0       0   
3             el       2       1       3       0       3       0       3   
4             es       1       1       2       0       1       2       1   
..           ...     ...     ...     ...     ...     ...     ...     ...   
370     especies       0       0       0       0       0       0       0   
371   amenazadas       0       0       0       0       0       0       0   
372  proporciona       0       0       0       0       0       0       0   
373    educativa       0       0       0       0       0       0       0   
374  entretenida       0       0       0       0       0       0       0   

     t10.txt  t9.txt  ...  t12.txt  t11.txt  t14.txt  t18.txt  t17.txt  \
0          0 